# Clean

In [2]:
from neo4j import GraphDatabase

# 连接到Neo4j数据库
uri = "bolt://localhost:7687"  # Neo4j数据库地址
username = "neo4j"  # 数据库用户名
password = "neo4j123456"  # 数据库密码

# 创建驱动
driver = GraphDatabase.driver(uri, auth=(username, password))

# 读取Cypher文件并执行查询
def execute_cypher_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        cypher_query = file.read()  # 读取Cypher文件内容
        
    # 执行查询
    with driver.session() as session:
        result = session.run(cypher_query)
        return result
    
def clear_database():
    clear_query = "MATCH (n) DETACH DELETE n"
    with driver.session() as session:
        session.run(clear_query)
    print("数据库已清空")

# 执行指定的Cypher文件
cypher_file = "./facilities/cypher/ynu_c.cypher"


clear_database()

#result = execute_cypher_file(cypher_file)

# 关闭驱动
driver.close()


数据库已清空


In [3]:
# from neo4j import GraphDatabase

# uri = "bolt://localhost:7687"
# user = "neo4j"
# password = "neo4j123456"

# driver = GraphDatabase.driver(uri, auth=(user, password))

# with driver.session() as session:
#     query = """
#     CALL apoc.export.cypher.all("exported_data26021nodes.cypher", {
#         format: 'cypher-shell',
#         useOptimizations: true
#     })
#     """
#     result = session.run(query)
#     print("导出完成，文件保存在Neo4j的import目录下")

# driver.close()

# 1.材料与能源

In [4]:
from neo4j import GraphDatabase
import json


# Neo4j数据库连接配置
URI = "bolt://localhost:7687"  # 替换实际地址
USER = "neo4j"                # 替换用户名
PASSWORD = "neo4j123456"    # 替换密码

def create_teachers(tx, teachers):
    """事务函数：创建教师节点并关联学院"""
    for teacher in teachers:
        # 执行Cypher语句
        tx.run("""
            MATCH (college:College {name: $college_name})
            MERGE (t:Person:Faculty {
                name: $name,
                position: $title,
                email: $email
            })
            MERGE (t)-[r:WORKS_IN]->(college)
            SET r.since = datetime()
        """, 
        college_name="材料与能源学院",  # 默认信息学院
        name=teacher["姓名"],
        title=teacher["职称"],
        email=teacher["联系方式"])

def main():
    # 从JSON文件加载数据
    with open("./teachers/clynyxy.json", "r", encoding="utf-8") as f:
        teachers = json.load(f)
    
    # 创建驱动实例
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    try:
        with driver.session() as session:
            # 执行写入事务
            session.execute_write(create_teachers, teachers)
            print(f"成功创建{len(teachers)}位教师节点")
    except Exception as e:
        print(f"操作失败：{str(e)}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()

成功创建86位教师节点


# info2vector

In [ ]:
# from neo4j import GraphDatabase
# from zhipuai import ZhipuAI
# import json
# import time

# def get_vector(batch):
#     client = ZhipuAI(api_key=ZHIPU_API_KEY)
#     response = client.embeddings.create(
#         model="embedding-3",
#         dimensions=1024,
#         input=batch
#     )
#     return response.data

# if __name__ == "__main__":
#     with open("./teachers/cxgjysxy.json", encoding="utf-8") as f:
#         teachers = json.load(f)

#     for i in range(0, len(teachers), 10):
#         teachers_intro = [teacher['简介'] for teacher in teachers[i:i+10]]
#         response = get_vector(teachers_intro)
#         if i == 0:
#             print(response[0].embedding)
#         assert len(response) == len(teachers_intro), f"API 返回数据不匹配: 期望 {len(teachers_intro)}, 得到 {len(response)}"
#         for j, teacher in enumerate(teachers[i:i+10]):
#             teacher["vector"] = response[j].embedding
#         print(f"已处理 {i+10}/{len(teachers)}")
#         time.sleep(1)
#     with open("./teachers/cxgjysxy_embedding.json", "w", encoding="utf-8") as f:
#         json.dump(teachers, f, ensure_ascii=False, indent=2)
    
    



# image2vec

### image download

In [6]:
# import json
# import os
# import re
# import requests
# from urllib.parse import urlparse
# import random

# def sanitize_filename(name):
#     """清理文件名中的非法字符"""
#     return re.sub(r'[\\/*?:"<>|]', '_', name)

# def download_works(teachers_data):
#     # 创建works目录
#     work_dir = "./teachers/works"
#     os.makedirs(work_dir, exist_ok=True)

#     for teacher in teachers_data:
#         name = sanitize_filename(teacher["姓名"])
#         works = teacher.get("作品", [])
#         work_paths = []

#         for idx, url in enumerate(works):
#             try:
#                 # 处理URL和文件名
#                 parsed = urlparse(url)
                
#                 # 获取原始扩展名
#                 original_filename = os.path.basename(parsed.path)
#                 _, ext = os.path.splitext(original_filename)
#                 if not ext:
#                     ext = ".jpg"  # 默认扩展名

#                 # 构建新文件名：姓名_序号 + 扩展名
#                 random_number = random.randint(10000000000, 99999999999)
#                 filename = f"{str(random_number)}_{idx}{ext}"
#                 save_path = os.path.join(work_dir, filename)

#                 # 下载文件
#                 response = requests.get(url, timeout=10)
#                 response.raise_for_status()

#                 # 保存文件
#                 with open(save_path, "wb") as f:
#                     f.write(response.content)
                
#                 work_paths.append(filename)

#             except Exception as e:
#                 print(f"下载失败: {url} - {str(e)}")
#                 continue

#         # 更新workPath字段
#         teacher["workPath"] = work_paths

#     return teachers_data

# def main():
#     # 读取原始数据
#     with open("./teachers/cxgjysxyandworks.json", "r", encoding="utf-8") as f:
#         data = json.load(f)

#     # 下载作品并更新数据
#     updated_data = download_works(data)

#     # 保存更新后的数据
#     with open("./cxgjysxy_updated.json", "w", encoding="utf-8") as f:
#         json.dump(updated_data, f, ensure_ascii=False, indent=2)

#     print("处理完成，更新后的数据已保存为 cxgjysxy_updated.json")

# if __name__ == "__main__":
#     main()

### image & text embedding

In [ ]:
# import dashscope
# import json
# import base64
# from http import HTTPStatus
# import os
# import requests
# import time



# def get_embedding(teachers):
#     """
#     读取教师数据，调用 Dashscope API 获取教师简介和作品的多模态嵌入向量，并更新到教师数据中
#     """
#     dashscope.api_key = os.environ['DASHSCOPE_API_KEY']

#     for teacher in teachers:
#         info = teacher.get('简介', '')
#         image_path = teacher.get('workPath', [])

#         # 处理文本嵌入
#         teacher['info_vector'] = None
#         teacher['works_vector'] = []

#         # resp = dashscope.MultiModalEmbedding.call(
#         #     model="multimodal-embedding-v1",
#         #     input=[{"text": info}]
#         # )
#         # if resp.status_code == HTTPStatus.OK and 'output' in resp:
#         #     teacher['info_vector'] = resp.output['embeddings'][0]['embedding']
#         # else:
#         #     print(f"获取文本嵌入失败: {resp.status_code}, {resp.text}")

#         # 处理图片嵌入
#         valid_paths = [os.path.abspath(os.path.join("./teachers/works", x))
#                        for x in image_path if os.path.exists(os.path.join("./teachers/works", x))]

#         if not valid_paths:
#             print(f"教师 {teacher.get('姓名', '未知')} 没有找到有效的图片路径")
#             continue
#         print(teacher['姓名'])
#         print(valid_paths)
        
#         image_vectors = []
#         for img_path in valid_paths:
#             try:
#                 with open(img_path, 'rb') as img_file:
#                     base64_image = base64.b64encode(img_file.read()).decode('utf-8')

#                 image_data = [{"image": f"data:image/jpg;base64,{base64_image}"}]
#                 resp = dashscope.MultiModalEmbedding.call(
#                     model="multimodal-embedding-v1",
#                     input=image_data
#                 )
#                 print(resp)
#                 if resp.status_code == HTTPStatus.OK and 'output' in resp:
#                     image_vectors.append(resp.output['embeddings'][0]['embedding'])
#                 else:
#                     print(f"获取图片嵌入失败: {resp.status_code}, {resp.text}")
#             except Exception as e:
#                 print(f"处理图片 {img_path} 时出错: {e}")

#             time.sleep(1)  # 避免 API 速率限制

#         teacher['works_vector'] = image_vectors
#         print(f"教师 {teacher.get('姓名', '未知')} 的图片嵌入已获取")

#     # 保存结果
#     with open("./teachers/cxgjysxy_modalEmbedding.json", "w", encoding="utf-8") as f:
#         json.dump(teachers, f, ensure_ascii=False, indent=2)


# def main():
#     # 读取 ./teachers/cxgjysxy_updated.json
#     with open("./teachers/cxgjysxy_updated.json", "r", encoding="utf-8") as f:
#         teachers = json.load(f)

#     get_embedding(teachers)


# if __name__ == '__main__':
#     main()


# 2昌新国际艺术学院

In [8]:
from py2neo import Graph, Node, Relationship
import json
import numpy as np

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 创建教师节点
                teacher_node = self.graph.nodes.match("Person", "Faculty").where(name=teacher_data["姓名"]).first()

                if teacher_node:
                    # 更新已有节点的 department 信息
                    teacher_node["department"] = teacher_data["系所"] + '系'
                else:
                    # 创建新节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        department=teacher_data["系所"] + '系'
                                    )
                    tx.create(teacher_node)


                # 创建并关联简介节点（含向量）
                if teacher_data["简介"]:
                    profile_node = Node("Profile",
                        text=teacher_data["简介"],
                        vector=teacher_data["info_vector"][0]
                    )
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                # 创建并关联作品节点（含URL和向量）
                for url, vector in zip(teacher_data["作品"], teacher_data["works_vector"]):
                    work_node = Node("ArtWork",
                        url=url,
                        vector=vector[0]
                    )
                    tx.create(work_node)
                    tx.create(Relationship(teacher_node, "HAS_WORK", work_node))

                # 关联到系所节点（假设系所节点已存在）
                query = """
                        MATCH (c:College {name: "昌新国际艺术学院"})-[:HAS_DEPARTMENT]->(d:Department)
                        WHERE d.name = $department_name
                        RETURN d
                        """
                department = self.graph.evaluate(query, parameters={"department_name": teacher_data["系所"]+'系'})
                
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department {teacher_data['系所']} not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/cxgjysxy_modalEmbedding.json")

Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\174741453.py:62: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 3.地球科学学院

In [9]:
from neo4j import GraphDatabase
import json

URI = "bolt://localhost:7687" 
USER = "neo4j"               
PASSWORD = "neo4j123456"    

def create_teachers(tx, teachers):
    """事务函数：创建教师节点并关联学院"""
    for teacher in teachers:
        # 执行Cypher语句
        tx.run("""
            MATCH (college:College {name: $college_name})
            MERGE (t:Person:Faculty {
                name: $name,
                position: $title,
                email: $email,
                research_field: $research_field
            })
            MERGE (t)-[r:WORKS_IN]->(college)
            SET r.since = datetime()
        """, 
        college_name="地球科学学院",
        name=teacher["姓名"],
        title=teacher["职称"],
        research_field=teacher["研究方向"],
        email=teacher["电子邮箱"])

def main():
    # 从JSON文件加载数据
    with open("./teachers/dqkxxy.json", "r", encoding="utf-8") as f:
        teachers = json.load(f)
    
    # 创建驱动实例
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    try:
        with driver.session() as session:
            # 执行写入事务
            session.execute_write(create_teachers, teachers)
            print(f"成功创建{len(teachers)}位教师节点")
    except Exception as e:
        print(f"操作失败：{str(e)}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()

成功创建39位教师节点


# 4.法学院

In [10]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        department=teacher_data["系所"] + '系',
                                        position=teacher_data["职称"]
                                    )
                    tx.create(teacher_node)
                else:
                    # 更新已有节点
                    teacher_node["department"] = teacher_data["系所"] + '系'
                    teacher_node["position"] = teacher_data["职称"]
                    tx.push(teacher_node)

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                # 创建并关联研究方向
                if teacher_data.get("研究方向"):
                    research_node = self.graph.nodes.match("ResearchArea", text=teacher_data["研究方向"]).first()
                    if not research_node:
                        research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                        tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                query = """
                        MATCH (c:College {name: "法学院"})-[:HAS_DEPARTMENT]->(d:Department)
                        WHERE d.name = $department_name
                        RETURN d
                        """
                department = self.graph.evaluate(query, parameters={"department_name": teacher_data["系所"]})
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department {teacher_data['系所']} not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/fxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2029328494.py:59: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 5.高等教育研究院

In [11]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        email=teacher_data["联系方式"],
                                        position=teacher_data["职称"],
                                        address=teacher_data["地址"]
                                    )
                    tx.create(teacher_node)
                

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                # 创建并关联研究方向
                if teacher_data.get("研究方向"):
                    research_node = self.graph.nodes.match("ResearchArea", text=teacher_data["研究方向"]).first()
                    if not research_node:
                        research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                        tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("Institute", name="高等教育研究院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/gdjyyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2024449869.py:49: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 6.国际关系研究院

In [12]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", ""),
                                    )
                    tx.create(teacher_node)

                # 创建并关联研究方向
                if teacher_data.get("研究方向"):
                    research_node = self.graph.nodes.match("ResearchArea", text=teacher_data["研究方向"]).first()
                    if not research_node:
                        research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                        tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("Institute", name="国际关系研究院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/gjgxxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4036769281.py:40: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 7.国际河流与生态安全研究院

In [13]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", ""),
                                    )
                    tx.create(teacher_node)


                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("Institute", name="国际河流与生态安全研究院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/gjhlstaqyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\1682594844.py:33: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 8.古生物研究院

In [14]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        email=teacher_data["联系方式"],
                                        position=teacher_data["职称"],
                                        address=teacher_data["地址"]
                                    )
                    tx.create(teacher_node)
                

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))



                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("Institute", name="古生物研究院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/gswyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2782538823.py:43: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 9.高原湖泊生态与治理研究院

In [15]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        email=teacher_data["邮箱"],
                                        position=teacher_data["职称"]
                                    )
                    tx.create(teacher_node)


                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("Institute", name="高原湖泊生态与治理研究院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/gyhpstyzlyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3885252037.py:34: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 10.化学科学与工程学院

In [16]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data["职称"]
                                    )
                    tx.create(teacher_node)


                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("College", name="化学科学与工程学院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/hxkxygcxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\328848580.py:33: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 11.汉语国际教育学院

In [17]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        email=teacher_data.get("联系方式", "")
                                    )
                    tx.create(teacher_node)
                

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

               
                # 关联到系所节点（假设系所节点已存在）
                college = self.graph.nodes.match("College", name="汉语国际教育学院").first()
                
                tx.create(Relationship(teacher_node, "BELONGS_TO", college))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/hygjjyxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\517138093.py:40: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 12.经济学院

In [18]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        address=teacher_data["办公地点"],
                                        position=teacher_data["职称"],
                                        email=teacher_data["联系邮箱"],
                                        office_hours=teacher_data["答疑时间"]
                                    )
                    tx.create(teacher_node)

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                # 创建并关联研究方向
                if teacher_data.get("研究方向") != "":
                    research_node = self.graph.nodes.match("ResearchArea", text=teacher_data["研究方向"]).first()
                    if not research_node:
                        research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                        tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))
                elif teacher_data.get("研究领域") != "":
                    research_node = self.graph.nodes.match("ResearchArea", text=teacher_data["研究方向"]).first()
                    if not research_node:
                        research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                        tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="经济学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/jjxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\73385129.py:57: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 13.自然资源药物化学教育部重点实验室

In [19]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        address=teacher_data["地址"],
                                        position=teacher_data["职称"],
                                        email=teacher_data["联系方式"]
                                    )
                    tx.create(teacher_node)
                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Laboratory", name="自然资源药物化学教育部重点实验室").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/jybzrzyywhxzdsys.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2540482328.py:35: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 14.建筑与规划学院

In [20]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data["职称"]
                                    )
                    tx.create(teacher_node)
                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="建筑与规划学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/jzyghxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\1125860595.py:33: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 15.历史与档案学院

In [21]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"]
                                    )
                    tx.create(teacher_node)

                # 创建并关联简介节点（含向量）
                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="历史与档案学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department {teacher_data['系所']} not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/lsydaxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3930139899.py:39: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 16.马克思主义学院

In [22]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("办公地点", "未知")
                                    )
                    tx.create(teacher_node)

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                if len(teacher_data.get("科研项目", [])) != 0:
                    for project in teacher_data["科研项目"]:
                        project_node = Node("Project", name=project['title'], date=project.get('date', '未知'))
                        tx.create(project_node)
                        tx.create(Relationship(teacher_node, "HAS_PROJECT", project_node))

                if len(teacher_data.get("专著", [])) != 0:
                    for book in teacher_data["专著"]:
                        book_node = Node("Book", name=book['title'], date=book.get('date', '未知'), publisher=book.get('publisher', '未知'))
                        tx.create(book_node)
                        tx.create(Relationship(teacher_node, "HAS_BOOK", book_node))
                
                if len(teacher_data.get("论文", [])) != 0:
                    for paper in teacher_data["论文"]:
                        paper_node = Node("Paper", name=paper['title'], date=paper.get('date', '未知'))
                        tx.create(paper_node)
                        tx.create(Relationship(teacher_node, "HAS_PAPER", paper_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="马克思主义学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/mkszyxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2037300877.py:60: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 17.民族学与政治学学院

In [23]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("办公地点", "未知")
                                    )
                    tx.create(teacher_node)

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                if len(teacher_data.get("科研项目", [])) != 0:
                    for project in teacher_data["科研项目"]:
                        project_node = Node("Project", name=project['title'], date=project.get('date', '未知'))
                        tx.create(project_node)
                        tx.create(Relationship(teacher_node, "HAS_PROJECT", project_node))

                if len(teacher_data.get("专著", [])) != 0:
                    for book in teacher_data["专著"]:
                        book_node = Node("Book", name=book['title'], date=book.get('date', '未知'), publisher=book.get('publisher', '未知'))
                        tx.create(book_node)
                        tx.create(Relationship(teacher_node, "HAS_BOOK", book_node))
                
                if len(teacher_data.get("论文", [])) != 0:
                    for paper in teacher_data["论文"]:
                        paper_node = Node("Paper", name=paper['title'], date=paper.get('date', '未知'))
                        tx.create(paper_node)
                        tx.create(Relationship(teacher_node, "HAS_PAPER", paper_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="民族学与社会学学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/mzxyshx.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4095736368.py:65: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 18.民族政治研究院

In [24]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("办公地点", "未知")
                                    )
                    tx.create(teacher_node)

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Institute", name="民族政治研究院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/mzzzyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3445502523.py:46: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 19.农学院

In [25]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("邮箱", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("邮箱", "未知")

                

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="农学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/nxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3919887404.py:39: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 20.软件学院

In [26]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("电子邮箱", "未知"),
                                        address=teacher_data.get("办公室", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("电子邮箱", "未知")
                    teacher_node["address"] = teacher_data.get("办公室", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                
                if teacher_data.get("研究领域"):
                    research_node = Node("ResearchArea", text=teacher_data["研究领域"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="软件学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/rjxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2145757641.py:50: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 21.生命科学学院

In [27]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="生命科学学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/smkxxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3158872468.py:43: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 22.生态与环境学院

In [28]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                if teacher_data["系所"] == "生物多样性研究院" or teacher_data["系所"] == "高原湖泊生态与治理研究院":
                    continue
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                query = """
                        MATCH (c:College {name: "生态与环境学院"})-[:HAS_DEPARTMENT]->(d:Department)
                        WHERE d.name = $department_name
                        RETURN d
                        """
                department = self.graph.evaluate(query, parameters={"department_name": teacher_data["系所"]})
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/styhjxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\317520585.py:48: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 23.生物多样性研究院

In [29]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("邮箱", "未知"),
                                        address=teacher_data.get("地址", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("邮箱", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
                    teacher_node["address"] = teacher_data.get("地址", "未知")
            

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="生物多样性研究院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/swdyxyjy.json")


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2410730196.py:39: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Knowledge graph built successfully!


# 24.生物医药研究院

In [30]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Institute", name="生物医药研究院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/swyyyjy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2523397079.py:42: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 25.数学与统计学院

In [31]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("电子邮件", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("电子邮件", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究领域"):
                    research_node = Node("ResearchArea", text=teacher_data["研究领域"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="数学与统计学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/sxytjxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3612253315.py:42: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 26.天文研究院

In [32]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("电子邮件", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Institute", name="中国西南天文研究所").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/twyjs.json")


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3102315802.py:45: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


Knowledge graph built successfully!


# 27.体育学院

In [33]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("电子邮件", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="体育学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/tyxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4256796876.py:42: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 28.外国语学院

In [34]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究领域及方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究领域及方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Department", name=teacher_data['系所']).first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/wgyxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4269508187.py:42: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 29.物理与天文学院

In [35]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="物理与天文学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/wlytwxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3188037250.py:35: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 30.文学院

In [36]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["title"] = teacher_data.get("职称", "未知")
            
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="文学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/wxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3883948920.py:39: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 31. "西南边疆少数民族研究中心",


In [37]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"]
                                    )
                    tx.create(teacher_node)

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Institute", name="西南边疆少数民族研究中心").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/xnbjssmzyjzx.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\1375037110.py:31: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 32.西南跨境生态安全重点实验室

In [38]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知")
                                    )
                    tx.create(teacher_node)

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Laboratory", name="西南跨境生态安全重点实验室").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/xnkjstaqjybzdsys.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4251389966.py:33: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 33.信息学院

In [39]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["名字"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["名字"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("地址", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("职称", "未知")
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["address"] = teacher_data.get("地址", "未知")

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                if len(teacher_data.get("科研项目", [])) != 0:
                    for project in teacher_data["科研项目"]:
                        project_node = Node("Project", name=project['title'], date=project.get('date', '未知'))
                        tx.create(project_node)
                        tx.create(Relationship(teacher_node, "HAS_PROJECT", project_node))

                if len(teacher_data.get("专著", [])) != 0:
                    for book in teacher_data["专著"]:
                        book_node = Node("Book", name=book['title'], date=book.get('date', '未知'), publisher=book.get('publisher', '未知'))
                        tx.create(book_node)
                        tx.create(Relationship(teacher_node, "HAS_BOOK", book_node))
                
                if len(teacher_data.get("专利", [])) != 0:
                    for patent in teacher_data["专利"]:
                        patent_node = Node("Patent", name=patent['title'], date=patent.get('date', '未知'))
                        tx.create(patent_node)
                        tx.create(Relationship(teacher_node, "HAS_PATENT", patent_node))
                
                if len(teacher_data.get("论文", [])) != 0:
                    for paper in teacher_data["论文"]:
                        paper_node = Node("Paper", name=paper['title'], date=paper.get('date', '未知'))
                        tx.create(paper_node)
                        tx.create(Relationship(teacher_node, "HAS_PAPER", paper_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="信息学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/xxxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\4016109974.py:75: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


In [40]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self):

        tx = self.graph.begin()
        teacher_node = self.graph.nodes.match("Person", "Faculty", name="金钊").first()
        
        department = self.graph.nodes.match("College", name="信息学院").first()
        if department:
            tx.create(Relationship(teacher_node, "BELONGS_TO", department))
            tx.commit()
        else:
            print(f"Warning: Department not found")


if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph()


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\1892202186.py:16: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 34.药学院

In [41]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("邮箱", "未知"),
                                        address=teacher_data.get("地址", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("职称", "未知")
                    teacher_node["email"] = teacher_data.get("邮箱", "未知")
                    teacher_node["address"] = teacher_data.get("地址", "未知")

                

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="药学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/yaoxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3653690898.py:42: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 35.一带一路研究院

In [42]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("职称", "未知")

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Institute", name="一带一路研究院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/ydylyjs.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3036078508.py:41: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 36.云南生物资源保护与利用国家重点实验室

In [43]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("地址", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("职称", "未知")
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["address"] = teacher_data.get("地址", "未知")

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))

                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("Laboratory", name="云南生物资源保护与利用国家重点实验室").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/ynswzybhylygjzdsys.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\1566134822.py:50: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 37.艺术与设计学院

In [44]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["name"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["name"],
                                        position=teacher_data.get("title", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("title", "未知")


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="艺术与设计学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/ysysjxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2148686822.py:37: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 38.医学院

In [45]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        title=teacher_data.get("职称", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["title"] = teacher_data.get("职称", "未知")

                if teacher_data.get("研究领域"):
                    research_node = Node("ResearchArea", text=teacher_data["研究领域"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="医学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/yxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\3634206899.py:41: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


# 39.政府管理学院

In [46]:
from py2neo import Graph, Node, Relationship
import json

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self.graph = Graph(uri, auth=(user, password))

    def build_graph(self, teachers_file):
        with open(teachers_file, 'r', encoding='utf-8') as f:
            teachers = json.load(f)

        tx = self.graph.begin()
        
        try:
            for teacher_data in teachers:
                # 检查教师节点是否已存在
                teacher_node = self.graph.nodes.match("Person", "Faculty", name=teacher_data["姓名"]).first()

                if not teacher_node:
                    # 创建新教师节点
                    teacher_node = Node("Person", "Faculty",
                                        name=teacher_data["姓名"],
                                        position=teacher_data.get("职称", "未知"),
                                        email=teacher_data.get("联系方式", "未知"),
                                        address=teacher_data.get("地址", "未知")
                                    )
                    tx.create(teacher_node)
                else:
                    teacher_node["position"] = teacher_data.get("职称", "未知")
                    teacher_node["email"] = teacher_data.get("联系方式", "未知")
                    teacher_node["address"] = teacher_data.get("地址", "未知")

                if teacher_data.get("个人简介"):
                    profile_node = Node("Profile", text=teacher_data["个人简介"])
                    tx.create(profile_node)
                    tx.create(Relationship(teacher_node, "HAS_PROFILE", profile_node))
                
                if teacher_data.get("研究方向"):
                    research_node = Node("ResearchArea", text=teacher_data["研究方向"])
                    tx.create(research_node)
                    tx.create(Relationship(teacher_node, "RESEARCHES", research_node))

                if len(teacher_data.get("科研项目", [])) != 0:
                    for project in teacher_data["科研项目"]:
                        project_node = Node("Project", name=project['title'], date=project.get('date', '未知'))
                        tx.create(project_node)
                        tx.create(Relationship(teacher_node, "HAS_PROJECT", project_node))

                if len(teacher_data.get("专著", [])) != 0:
                    for book in teacher_data["专著"]:
                        book_node = Node("Book", name=book['title'], date=book.get('date', '未知'), publisher=book.get('publisher', '未知'))
                        tx.create(book_node)
                        tx.create(Relationship(teacher_node, "HAS_BOOK", book_node))
                
                if len(teacher_data.get("论文", [])) != 0:
                    for paper in teacher_data["论文"]:
                        paper_node = Node("Paper", name=paper['title'], date=paper.get('date', '未知'))
                        tx.create(paper_node)
                        tx.create(Relationship(teacher_node, "HAS_PAPER", paper_node))


                # 关联到系所节点（假设系所节点已存在）
                department = self.graph.nodes.match("College", name="政府管理学院").first()
                if department:
                    tx.create(Relationship(teacher_node, "BELONGS_TO", department))
                else:
                    print(f"Warning: Department not found")

            tx.commit()
            print("Knowledge graph built successfully!")
            
        except Exception as e:
            tx.rollback()
            print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 配置数据库连接
    kg_builder = KnowledgeGraphBuilder(
        uri="bolt://localhost:7687", 
        user="neo4j",
        password="neo4j123456"
    )
    
    # 构建知识图谱
    kg_builder.build_graph("./teachers/zfglxy.json")


Knowledge graph built successfully!


C:\Users\Burning\AppData\Local\Temp\ipykernel_16760\2443116881.py:69: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()
